In [1]:
import torchvision
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
devive = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

train_data = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=torchvision.transforms.ToTensor())

test_data = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=torchvision.transforms.ToTensor())

train_data_size = len(train_data)
test_data_size = len(test_data)

print("训练集大小：", train_data_size)
print("测试集大小：", test_data_size)

# 利用 DataLoader 来加载数据集
train_dataloader = DataLoader(train_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

# CIFAR10 Model
class CIFAR10Model(nn.Module):
    def __init__(self):
        super(CIFAR10Model, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, 5, stride=1, padding=2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 32, 5, stride=1, padding=2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 5, stride=1, padding=2),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(4*4*64, 64),
            nn.Linear(64, 10)
        )
    
    def forward(self, x):
        return self.model(x)


Files already downloaded and verified
Files already downloaded and verified
训练集大小： 50000
测试集大小： 10000


In [2]:
model = CIFAR10Model().to(devive)
# model = model.cuda()

# 学习率
learning_rate = 0.001

# 定义损失函数和优化器
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# 训练模型

# 记录训练的次数
total_train_step = 0

# 记录测试的次数
total_test_step = 0

# 添加tensorboard
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter("./logs/logs_train")

# 训练的轮数
epoch = 30

# 训练
for i in range(epoch):
    print("-----------第{}轮训练-----------".format(i+1))
    # 训练步骤开始
    model.train()
    for data in train_dataloader:
        imgs, targets = data
        imgs, targets = imgs.to(devive), targets.to(devive)
        outputs = model(imgs)
        loss = loss_fn(outputs, targets)

        # 反向传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_train_step += 1
        if total_train_step % 1000 == 0:
            print("训练次数：{}, Loss: {}".format(total_train_step, loss.item()))
            writer.add_scalar("train_loss", loss.item(), total_train_step)
    # 测试
    model.eval()
    with torch.no_grad():
        total = 0
        correct = 0
        total_test_loss = 0
        
        for data in test_dataloader:
            imgs, targets = data
            imgs, targets = imgs.to(devive), targets.to(devive)
            outputs = model(imgs)
            loss = loss_fn(outputs, targets)
            total_test_loss += loss.item()
            
            # 获取预测的分类
            _, predicted = torch.max(outputs.data, 1)
            
            total += targets.size(0)
            correct += (predicted == targets).sum()
            
            total_test_step += 1
            
            if total_test_step % 100 == 0:
                print("测试次数：{}, Loss: {}".format(total_test_step, total_test_loss / total_test_step))
                print("测试次数：{}, 准确率：{}".format(total_test_step, (correct / total).item()))
                writer.add_scalar("test_acc", (correct / total).item(), total_test_step)
                writer.add_scalar("test_loss", total_test_loss / total_test_step, total_test_step)
    if i % 6 == 0:
        torch.save(model.state_dict(), "./Models/CIFAR10Model_{}.pth".format(i+1))

writer.close()



-----------第1轮训练-----------
测试次数：100, Loss: 1.1834237617254257
测试次数：100, 准确率：0.5901562571525574
-----------第2轮训练-----------
训练次数：1000, Loss: 1.2527399063110352
测试次数：200, Loss: 0.2255658021569252
测试次数：200, 准确率：0.635901153087616
测试次数：300, Loss: 0.49877717475096384
测试次数：300, 准确率：0.6381118893623352
-----------第3轮训练-----------
训练次数：2000, Loss: 1.2172940969467163
测试次数：400, Loss: 0.21829635113477708
测试次数：400, 准确率：0.6589753031730652
-----------第4轮训练-----------
训练次数：3000, Loss: 0.8594892621040344
测试次数：500, Loss: 0.056205348253250124
测试次数：500, 准确率：0.6740301847457886
测试次数：600, Loss: 0.2115958927075068
测试次数：600, 准确率：0.6701792478561401
-----------第5轮训练-----------
测试次数：700, Loss: 0.10329586608069284
测试次数：700, 准确率：0.6720920205116272
-----------第6轮训练-----------
训练次数：4000, Loss: 0.7768449783325195
测试次数：800, Loss: 0.020015108063817025
测试次数：800, 准确率：0.6656250357627869
测试次数：900, Loss: 0.13481982707977294
测试次数：900, 准确率：0.6648097634315491
-----------第7轮训练-----------
训练次数：5000, Loss: 0.7834130525588989
测试次数：

In [153]:
from PIL import Image

image_path = "./img/dog.png"
img = Image.open(image_path)

# 对图像进行预处理
transform = torchvision.transforms.Compose([torchvision.transforms.Resize((32, 32)), torchvision.transforms.ToTensor()])

image = transform(img)
image = torch.reshape(image, (-1, 3, 32, 32)).cuda()
# print(image.shape)

# model_ = torch.load("./Models/CIFAR10Model_3.pth")


model_ = CIFAR10Model().cuda()
model_.load_state_dict(torch.load('./Models/CIFAR10Model_3.pth'))
# model_.load_state_dict(torch.load("./Models/CIFAR10Model_6.pth"))

# model = TheModelClass(*args, **kwargs)
# model.load_state_dict(torch.load(PATH))
# model.eval()


model_.eval()
# print(model_)

with torch.no_grad():
    output = model_(image)
    print(output)
    _, predicted = torch.max(output.data, 1)
    print(predicted.item())
    print("预测结果：", train_data.classes[predicted.item()])

tensor([[-2.1253, -5.6033,  2.5014,  2.7626,  3.4610,  4.3226, -4.0161,  5.1608,
         -4.4436, -2.2987]], device='cuda:0')
7
预测结果： horse
